In [19]:
from scipy.special import comb
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

 Stabilizer of the form $$I^{\otimes dI}\otimes X^{\otimes dX}\otimes Y^{\otimes dY}\otimes Z^{\otimes dZ}$$
 Each error pattern weight distribution can be expressed as four weight distributions within the intervals $[0,dI), [dI,dI+dX), [dI+dX,dI+dX+dY), [dI+dX+dY,dI+dX+dY+dZ)$.

In [44]:
class wgt_dist:
    def __init__(self, dI, dX, dY, dZ):
        self.dI = dI
        self.dX = dX
        self.dY = dY
        self.dZ = dZ
    def display(self):
        print('I: ',self.dI,'X: ',self.dX,'Y: ',self.dY,'Z: ',self.dZ)
    def num_Pauli(self, Pauli):
        if Pauli == 'I':
            return self.dI
        elif Pauli == 'X':
            return self.dX
        elif Pauli == 'Y':
            return self.dY
        elif Pauli == 'Z':
            return self.dZ
        else:
            raise ValueError('Pauli must be I, X, Y, or Z')
    def apply_Pauli(self,Pauli):
        if Pauli == 'I':
            return wgt_dist(self.dI, self.dX, self.dY, self.dZ)
        elif Pauli == 'X':
            return wgt_dist(self.dX, self.dI, self.dZ, self.dY)
        elif Pauli == 'Y':
            return wgt_dist(self.dY, self.dZ, self.dI, self.dX)
        elif Pauli == 'Z':
            return wgt_dist(self.dZ, self.dY, self.dX, self.dI)
        else:
            raise ValueError('Pauli must be I, X, Y, or Z')
        
class error_wgt_dist:
    def __init__(self, wgt_dist_I_interval, wgt_dist_X_interval, wgt_dist_Y_interval, wgt_dist_Z_interval):
        
        self.wgt_dist_I_interval = wgt_dist_I_interval
        self.wgt_dist_X_interval = wgt_dist_X_interval
        self.wgt_dist_Y_interval = wgt_dist_Y_interval
        self.wgt_dist_Z_interval = wgt_dist_Z_interval
        self.all_wgts = [self.wgt_dist_I_interval, self.wgt_dist_X_interval, self.wgt_dist_Y_interval, self.wgt_dist_Z_interval]
        
    def num_Pauli(self, Pauli):
        # Returns the number of Pauli errors of type Pauli within the error
        weight = 0
        
        for wgt_dist in self.all_wgts:
            weight += wgt_dist.num_Pauli(Pauli)
        return weight
    
    def error_wgt_dist_plus_stabilizer(self):
        return error_wgt_dist(self.wgt_dist_I_interval,
                              self.wgt_dist_X_interval.apply_Pauli('X'), 
                              self.wgt_dist_Y_interval.apply_Pauli('Y'), 
                              self.wgt_dist_Z_interval.apply_Pauli('Z'))
    
     
            

In [45]:
def entropy(p):
    e = 0
    for pi in p:
        if pi !=0:
            e =e-pi*np.log2(pi)
    return e

def binary_entropy(p):
    if p == 0 or p == 1:
        return 0
    else:
        return -p*np.log2(p)-(1-p)*np.log2(1-p)

In [46]:
def probability_of_error(error_wgt_dist,pX,pY,pZ):
    pI = 1-pX-pY-pZ
    p = [pI,pX,pY,pZ]
    Paulis = ['I','X','Y','Z']
    prob = 1
    for i in range(4):
        prob *= p[i]**(error_wgt_dist.num_Pauli(Paulis[i]))
    return prob
    


In [72]:
def all_possible_wgt_dists_within_interval(d):

    wgt_dists = []
    multiplier = []
    for kI in range(d+1):
        for kX in range(d+1-kI):
            for kY in range(d+1-kI-kX):
                kZ = d-kI-kX-kY
                wgt_dists.append(wgt_dist(kI,kX,kY,kZ))
                multiplier.append(comb(d,kI)*comb(d-kI,kX)*comb(d-kI-kX,kY))
    return wgt_dists,multiplier

In [73]:
w,m = all_possible_wgt_dists_within_interval(2)


In [84]:
def joint_degeneracy_entropy_post_mmt(Stabilizer_dI,Stabilizer_dX,Stabilizer_dY,Stabilizer_dZ,pX,pY,pZ):
    wI, mI =  all_possible_wgt_dists_within_interval(Stabilizer_dI)
    wX, mX =  all_possible_wgt_dists_within_interval(Stabilizer_dX)
    wY, mY =  all_possible_wgt_dists_within_interval(Stabilizer_dY)
    wZ, mZ =  all_possible_wgt_dists_within_interval(Stabilizer_dZ)
    
    entropy = 0 
    
    for i in range(len(wI)):
        for j in range(len(wX)):
            for k in range(len(wY)):
                for l in range(len(wZ)):
                    wgt_dist_I_interval = wI[i]
                    wgt_dist_X_interval = wX[j]
                    wgt_dist_Y_interval = wY[k]
                    wgt_dist_Z_interval = wZ[l]
                    error_wgt_dist_interval = error_wgt_dist(wgt_dist_I_interval, wgt_dist_X_interval, wgt_dist_Y_interval, wgt_dist_Z_interval)
                    error_wgt_dist_plus_stabilizer_interval = error_wgt_dist_interval.error_wgt_dist_plus_stabilizer()
                    pE = probability_of_error(error_wgt_dist_interval,pX,pY,pZ)
                    pE_plus_stabilizer = probability_of_error(error_wgt_dist_plus_stabilizer_interval,pX,pY,pZ)
                    ptot = pE+pE_plus_stabilizer
                    if ptot != 0:
                        entropy+= -1*mI[i]*mX[j]*mY[k]*mZ[l]*ptot*np.log2(ptot)
                        
    entropy = 0.5*entropy
    return entropy


In [85]:
def syndrome_probability(dI,dX,dY,dZ,pX,pY,pZ): 
    # dI assumed  = 0 
    return (1/2) *(1 - (1 - 2*pY - 2*pZ )**dX *(1 - 2*pX - 2*pZ )**dY* (1 - 2*pX - 2*pY )**dZ)

In [86]:
joint_degeneracy_entropy_post_mmt(Stabilizer_dI=0,Stabilizer_dX=2,Stabilizer_dY=0,Stabilizer_dZ=2,pX=0.1,pY=0.1,pZ=0.1)

5.229613015646783

In [87]:
def rate_improvement_possible(Stabilizer_dI,Stabilizer_dX,Stabilizer_dY,Stabilizer_dZ,pX,pY,pZ):
    #dI assumed  = 0
    #Delta = nh(p) − 1 − H(L, Ssyn) + H(Ssyn) 
    n = Stabilizer_dI+Stabilizer_dX+Stabilizer_dY+Stabilizer_dZ
    Delta = n*entropy([1-pX-pY-pZ,pX,pY,pZ]) - 1 - joint_degeneracy_entropy_post_mmt(Stabilizer_dI,Stabilizer_dX,Stabilizer_dY,Stabilizer_dZ,pX,pY,pZ) + binary_entropy(syndrome_probability(Stabilizer_dI,Stabilizer_dX,Stabilizer_dY,Stabilizer_dZ,pX,pY,pZ))
    if Delta > 0:
        return True
    else:
        return False

In [94]:
rate_improvement_possible(0,9,0,0,0.1,0,0)

False

$$h(p)-\delta(p),$$
$$\delta(p) = n h(p) + H(S_{syn}) - H(S_{syn},L)$$

In [107]:
pp = 0.19
delta = 0.000000000001
channel = [1 - pp, pp/2 - delta/2, pp/2 - delta/2, delta]
pI = channel[0]
pX = channel[1]
pY = channel[2]
pZ = channel[3]


In [115]:

dI = 0
dX = 2
dY = 3
dZ = 2
n = dI+dX+dY+dZ
hp = entropy([1-pX-pY-pZ,pX,pY,pZ])
delta_p = n*hp-joint_degeneracy_entropy_post_mmt(dI,dX,dY,dZ,pX,pY,pZ)+binary_entropy(syndrome_probability(dI,dX,dY,dZ,pX,pY,pZ))
print('h(p)-δ(p) = ',hp-delta_p, 'rate is non-zero only if h(p) < δ(p), i.e., h(p)-δ(p) < 0')

h(p)-δ(p) =  -0.0966005284097079 rate is non-zero only if h(p) < δ(p), i.e., h(p)-δ(p) < 0


In [109]:
hashing = 1-hp

In [110]:
hashing

0.10852854007819268

In [111]:
binary_entropy(syndrome_probability(dI,dX,dY,dZ,pX,pY,pZ))

0.9870022172670929

In [114]:
reduction_due_to_degeneracy = n*hp-joint_degeneracy_entropy_post_mmt(0,2,3,2,pX,pY,pZ)
print(reduction_due_to_degeneracy)

0.001069771064422298
